In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    policy_df = spark.table("genai_demo.jnj.policy")
    claims_df = spark.table("genai_demo.jnj.claims")
    demographics_df = spark.table("genai_demo.jnj.demographics")
    scores_df = spark.table("genai_demo.jnj.scores")
    aiml_insights_df = spark.table("genai_demo.jnj.aiml_insights")

    # Log the schema of each DataFrame for validation
    logger.info("Policy DataFrame Schema: %s", policy_df.schema)
    logger.info("Claims DataFrame Schema: %s", claims_df.schema)
    logger.info("Demographics DataFrame Schema: %s", demographics_df.schema)
    logger.info("Scores DataFrame Schema: %s", scores_df.schema)
    logger.info("AI/ML Insights DataFrame Schema: %s", aiml_insights_df.schema)

    # Select relevant fields from each DataFrame
    demographics_df = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City", "State", "Postal_Code",
        "Date_of_Birth", "Gender", "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )

    claims_df = claims_df.select(
        "Claim_ID", "Policy_ID", "Claim_Date", "Claim_Type", "Claim_Status", "Claim_Amount", "Claim_Payout"
    )

    policy_df = policy_df.select(
        "Policy_ID", "Customer_ID", "Policy_Type", "Policy_Status", "Policy_Start_Date", "Policy_End_Date",
        "Policy_Term", "Policy_Premium", "Total_Premium_Paid", "Renewal_Status", "Policy_Addons"
    )

    scores_df = scores_df.select(
        "Customer_ID", "Credit_Score", "Fraud_Score", "Customer_Risk_Score"
    )

    aiml_insights_df = aiml_insights_df.select(
        "Customer_ID", "Churn_Probability", "Next_Best_Offer", "Claims_Fraud_Probability", "Revenue_Potential"
    )

    # Join demographics and policy data on Customer_ID
    demographics_policy_df = demographics_df.join(policy_df, "Customer_ID", "inner")

    # Join claims and policy data on Policy_ID
    claims_policy_df = claims_df.join(policy_df, "Policy_ID", "inner")

    # Summarize claims data
    summarized_claims_df = claims_policy_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.count("Policy_ID").alias("Policy_Count"),
        F.max("Claim_Date").alias("Recent_Claim_Date"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount")
    )

    # Join summarized data with combined data on Customer_ID
    combined_df = demographics_policy_df.join(summarized_claims_df, "Customer_ID", "inner")

    # Custom Calculations
    combined_df = combined_df.withColumn(
        "Age", F.floor(F.datediff(F.current_date(), F.to_date("Date_of_Birth", "yyyy-MM-dd")) / 365)
    ).withColumn(
        "Claim_To_Premium_Ratio", F.when(F.col("Total_Premium_Paid") != 0, F.col("Average_Claim_Amount") / F.col("Total_Premium_Paid")).otherwise(0)
    ).withColumn(
        "Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)
    ).withColumn(
        "Retention_Rate", F.lit(0.85)
    ).withColumn(
        "Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")
    ).withColumn(
        "Upsell_Potential", F.lit("Premium Vehicle Coverage")
    )

    # Join with AI/ML insights and scores data
    final_df = combined_df.join(aiml_insights_df, "Customer_ID", "inner").join(scores_df, "Customer_ID", "inner")

    # Drop existing table if necessary
    spark.sql("DROP TABLE IF EXISTS genai_demo.jnj.customer_360")

    # Write the final DataFrame to Unity Catalog table
    final_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.jnj.customer_360")

    logger.info("ETL process completed successfully and data written to genai_demo.jnj.customer_360")

except Exception as e:
    logger.error(f"An error occurred during the ETL process: {e}")
    # Additional logging for debugging
    logger.debug("Exception details:", exc_info=True)
